In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
genome_millepora = '/content/drive/MyDrive/GCF_013753865.1_Amil_v2.1_genomic.fna'
protein = '/content/drive/MyDrive/protein.faa'
genomic = '/content/drive/MyDrive/genomic.gff'

In [ ]:
# Установка HMMER
!wget http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz
!tar xf hmmer-3.3.2.tar.gz
%cd hmmer-3.3.2
!./configure && make && make install
%cd ..

# Скачивание базы Pfam
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz
!gunzip Pfam-A.hmm.gz
!hmmpress Pfam-A.hmm

--2025-06-12 12:45:30--  http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz
Resolving eddylab.org (eddylab.org)... 96.126.110.11, 2600:3c03::f03c:91ff:fec8:383c
Connecting to eddylab.org (eddylab.org)|96.126.110.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18213049 (17M) [application/x-gzip]
Saving to: ‘hmmer-3.3.2.tar.gz’

hmmer-3.3.2.tar.gz  100%[===================>]  17.37M  36.9MB/s    in 0.5s    

2025-06-12 12:45:31 (36.9 MB/s) - ‘hmmer-3.3.2.tar.gz’ saved [18213049/18213049]

/content/hmmer-3.3.2
configure: Configuring HMMER3 for your system.
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking whether to compile using MPI... no
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking wheth

In [ ]:
!hmmsearch --tblout search.csv Pfam-A.hmm "$protein"

Выходные данные были обрезаны до нескольких последних строк (5000).
  Alignments for each domain:
  == domain 1  score: 175.2 bits;  conditional E-value: 6.3e-54
                      E-SSEEEEEEESSTTSS----GGGEEETT..TTB--SEEETTTTEEEEEEETTSTT-EEE..E-SSEEEEEEEEEE-----SST.T..EESS- CS
  Zona_pellucida    1 CgedkmtvsvsksllqgsglnpsslslndsdpsCratantattaevvfefplsgCGttvkeeseddkliysnelvaqpesssevtddeiarrs 93  
                      C+++ m++s++++++  +g+np++l+l    +sCra+ n  +t +++++++l++CGtt++  +++d +++ ne+++ + s+++v    i+r++
  XP_044184607.1 1074 CTDNAMSISLERKSF--KGFNPNTLTLLY--SSCRASYN--DT-HITLRTSLNDCGTTHN--ESEDAITFYNEVRSVQFSPGTV----ITREQ 1153
                      9**************..99********99..****8854..33.469**********997..78***********866555444....45555 PP

                      -EEEEEEEEEE-----.................------EEEEEE-TEEB.............-TT-EEEEEEEE--TS--EEEEEEEEEEES CS
  Zona_pellucida   94 itrlkfsCtYprnesvslssai..se.vlavk.aegsftvsmrlftdssysspy.sedypvevllgdplyvevsvnsrtpnlelfldsCwatp 181 
 

In [ ]:
# Проверим содержимое
with open("epigenetic_domains.txt", "r") as f:
    domains = [line.strip() for line in f if line.strip()]
print("Загружено доменов:", len(domains))
print(domains[:10])  # первые 10 доменов для проверки

Загружено доменов: 13
['PF00145', 'PF00385', 'PF00856', 'PF00583', 'PF02184', 'PF01429', 'PF02146', 'PF00855', 'PF00505', 'PF02373']


In [ ]:
def read_domains(file_name):
    """Считываем Pfam-домены из файла и возвращаем как множество."""
    with open(file_name, 'r') as f:
        return {line.strip() for line in f if line.strip()}

def filter_tblout(tblout_file, domains_file, output_file):
    """Фильтруем hmmsearch-результаты по списку Pfam-доменов."""
    domains = read_domains(domains_file)

    with open(tblout_file, 'r') as inp, open(output_file, 'w') as out:
        for line in inp:
            if line.startswith('#'):
                continue
            fields = line.strip().split()
            if fields and fields[0] in domains:
                out.write(line)

# Запуск фильтрации
filter_tblout("search.csv", "epigenetic_domains.txt", "filtered_epigenetic_hits.csv")
print("Готово: отфильтрованные строки записаны в filtered_epigenetic_hits.csv")


Готово: отфильтрованные строки записаны в filtered_epigenetic_hits.csv


In [ ]:
import csv

def create_gene_family_table(filtered_file, output_file):
    with open(filtered_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['Pfam_Domain', 'Protein_ID'])

        for line in f_in:
            if line.startswith('#'): continue
            parts = line.strip().split()
            if len(parts) >= 3:
                pfam = parts[0]
                protein = parts[2]
                writer.writerow([pfam, protein])

# Создание таблицы
create_gene_family_table("filtered_epigenetic_hits.csv", "gene_family_table.csv")
print("Готово: таблица Pfam-домен – белок сохранена как gene_family_table.csv")


Готово: таблица Pfam-домен – белок сохранена как gene_family_table.csv


In [ ]:
def remove_duplicates(input_csv, output_csv):
    seen = set()
    with open(input_csv, 'r', newline='') as inp, open(output_csv, 'w', newline='') as out:
        reader = csv.reader(inp)
        writer = csv.writer(out)
        header = next(reader)
        writer.writerow(header)

        for row in reader:
            key = tuple(row)
            if key not in seen:
                seen.add(key)
                writer.writerow(row)

remove_duplicates("gene_family_table.csv", "gene_family_table_unique.csv")
print("Готово: уникальные записи записаны в gene_family_table_unique.csv")


Готово: уникальные записи записаны в gene_family_table_unique.csv


In [ ]:
# Посмотрим первые строки файла search.csv
with open("search.csv", "r") as f:
    for _ in range(10):
        print(f.readline().strip())

#                                                               --- full sequence ---- --- best 1 domain ---- --- domain number estimation ----
# target name        accession  query name           accession    E-value  score  bias   E-value  score  bias   exp reg clu  ov env dom rep inc description of target
#------------------- ---------- -------------------- ---------- --------- ------ ----- --------- ------ -----   --- --- --- --- --- --- --- --- ---------------------
XP_029179874.1       -          1-cysPrx_C           PF10417.12   1.7e-14   54.5   0.1   5.7e-14   52.9   0.2   1.8   2   0   0   2   2   2   1 peroxiredoxin 1-like [Acropora millepora]
XP_029188849.1       -          1-cysPrx_C           PF10417.12   6.2e-13   49.5   0.0   1.1e-12   48.7   0.0   1.4   1   0   0   1   1   1   1 peroxiredoxin-like isoform X2 [Acropora millepora]
XP_029188848.1       -          1-cysPrx_C           PF10417.12   8.9e-13   49.0   0.0   1.5e-12   48.3   0.0   1.4   1   0   0   1   1   1   1

In [ ]:
def read_domains(file_name):
    """Считываем Pfam-домены без версии, например PF00145 из PF00145.12"""
    return {line.strip().split('.')[0] for line in open(file_name) if line.strip()}

def filter_tblout(tblout_file, domains_file, output_file):
    """Фильтруем строки по Pfam accession (без .12)"""
    domains = read_domains(domains_file)

    with open(tblout_file, 'r') as inp, open(output_file, 'w') as out:
        for line in inp:
            if line.startswith('#'):
                continue
            fields = line.strip().split()
            if len(fields) > 3:
                pfam_accession = fields[3].split('.')[0]
                if pfam_accession in domains:
                    out.write(line)


In [ ]:
import csv

def create_gene_family_table(filtered_file, output_file):
    with open(filtered_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['Pfam_Domain', 'Protein_ID'])

        for line in f_in:
            if line.startswith('#'): continue
            parts = line.strip().split()
            if len(parts) >= 4:
                pfam = parts[3].split('.')[0]
                protein = parts[0]
                writer.writerow([pfam, protein])


In [ ]:
# Список всех Pfam-доменов, найденных в search.csv (без версии)
pfams_in_search = set()

with open("search.csv", "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        parts = line.strip().split()
        if len(parts) > 3:
            pfam = parts[3].split(".")[0]  # обрезаем .12
            pfams_in_search.add(pfam)

print("Всего уникальных Pfam-доменов в search.csv:", len(pfams_in_search))
print("Примеры:", list(pfams_in_search)[:10])


Всего уникальных Pfam-доменов в search.csv: 12877
Примеры: ['PF06745', 'PF05597', 'PF14592', 'PF20400', 'PF13963', 'PF07096', 'PF12971', 'PF18261', 'PF17877', 'PF14114']


In [ ]:
# Сравним домены в search.csv с твоими из epigenetic_domains.txt
with open("epigenetic_domains.txt", "r") as f:
    my_domains = {line.strip() for line in f if line.strip()}

common = my_domains.intersection(pfams_in_search)

print("Совпадающих Pfam-доменов:", len(common))
print("Общие домены:", common)


Совпадающих Pfam-доменов: 13
Общие домены: {'PF00505', 'PF02184', 'PF00385', 'PF00855', 'PF02373', 'PF00583', 'PF00651', 'PF01429', 'PF00856', 'PF02146', 'PF00439', 'PF00145', 'PF00244'}


In [ ]:
def filter_tblout(tblout_file, domains_file, output_file):
    domains = {line.strip().split('.')[0] for line in open(domains_file) if line.strip()}

    with open(tblout_file, 'r') as inp, open(output_file, 'w') as out:
        for line in inp:
            if line.startswith('#'):
                continue
            parts = line.strip().split()
            if len(parts) > 3:
                pfam = parts[3].split('.')[0]
                if pfam in domains:
                    out.write(line)


In [ ]:
filter_tblout("search.csv", "epigenetic_domains.txt", "filtered_epigenetic_hits.csv")
print("✅ Отфильтровано!")


✅ Отфильтровано!


In [ ]:
import csv

def create_gene_family_table(filtered_file, output_file):
    with open(filtered_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(['Pfam_Domain', 'Protein_ID'])

        for line in f_in:
            if line.startswith('#'): continue
            parts = line.strip().split()
            if len(parts) >= 4:
                pfam = parts[3].split('.')[0]
                protein = parts[0]
                writer.writerow([pfam, protein])

create_gene_family_table("filtered_epigenetic_hits.csv", "gene_family_table.csv")
print("✅ Таблица готова: gene_family_table.csv")


✅ Таблица готова: gene_family_table.csv
